# Imports

In [1]:
import os
from argparse import Namespace

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration
from seqeval.metrics import f1_score, classification_report
import pytorch_lightning as pl

In [3]:
from src.data.make_conll2003 import get_example_sets, InputExample
from src.models.modeling_t5conll2003 import T5ForConll2003

In [4]:
hparams = {
    "experiment_name": "Overfit T5 on CoNLL2003",
    "batch_size": 2, "num_workers": 2,
    "optimizer": "Adam", "lr": 5e-3,
    "datapath": "../data/conll2003",
    "shuffle_train": False,
    "source_max_length": 128,
    "target_max_length": 256,
    "labels_mode": 'tokens',
    "merge_O": True,
#     "token_weights": (
#        ('<O>', 0.1),
#     )
}
hparams = Namespace(**hparams)

In [5]:
model = T5ForConll2003.from_pretrained('t5-small', hparams=hparams)

# Overfit and save

In [6]:
overfit = True
overfit_ckpt = 'overfit_tokens_mode.ckpt'

In [7]:
model.prepare_data()
dl_train = model.train_dataloader()
batch = next(iter(dl_train))

In [8]:
if overfit:
    device = 'cuda'

    model.to(device)
    batch = [x.to(device) for x in batch]

    optimizer = model.configure_optimizers()

    for _ in range(50):
        loss = model.training_step(batch, 0)['loss']
        loss.backward()
        optimizer.step()
        model.zero_grad()

        print(loss.item())

    torch.save(model.state_dict(), overfit_ckpt)
else:
    print(model.load_state_dict(torch.load(overfit_ckpt)))

/opt/conda/conda-bld/pytorch_1587428207430/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


13.251261711120605
11.20366096496582
8.816116333007812
6.981232166290283
5.593908786773682
4.384148120880127
3.5685553550720215
2.8973278999328613
1.8214129209518433
1.6491810083389282
1.582523226737976
1.4420469999313354
1.3759634494781494
1.4876009225845337
1.4521602392196655
1.2886043787002563
1.2738251686096191
1.25042724609375
1.2025346755981445
1.1466948986053467
1.0892550945281982
1.026159405708313
0.9719151258468628
0.9187763333320618
0.8655423521995544
0.8059073686599731
0.7518723607063293
0.6992502808570862
0.6488634347915649
0.5981446504592896
0.5494703650474548
0.5029447674751282
0.4588232934474945
0.41660574078559875
0.37697726488113403
0.3407685160636902
0.3074922263622284
0.2761213183403015
0.24776655435562134
0.22275099158287048
0.20037338137626648
0.1803586333990097
0.16294415295124054
0.14796634018421173
0.13486717641353607
0.12353809922933578
0.11369551718235016
0.10496120899915695
0.09715239703655243
0.0900764912366867


# Evaluation

In [9]:
tokenizer = model.tokenizer

In [10]:
entities_tokens = model.entities_tokens
entities_tokens

['<O>', '<PER>', '<ORG>', '<LOC>', '<MISC>']

In [11]:
target_token_ids = batch[2].cpu()
target_token_ids = target_token_ids.where(target_token_ids != -100, torch.tensor(tokenizer.pad_token_id)).cpu()

In [12]:
predicted_token_ids = model.generate(input_ids=batch[0], attention_mask=batch[1], max_length=model.max_length)

In [13]:
tokenizer.decode(target_token_ids[0])

'EU <ORG> rejects <O> German <MISC> call to boycott <O> British <MISC> lamb. <O> '

In [14]:
tokenizer.decode(predicted_token_ids[0])

'EU <ORG> rejects <O> German <MISC> call to boycott <O> British <MISC> lamb. <O> '

In [22]:
def get_entities_from_tokens(tokens, tokenizer, entities_tokens, length=0, fill_token='O'):
    sequence_entities = [] # will save all the entities
    current_entity = [] # will save current entity
    if tokens[0] == tokenizer.pad_token:
        tokens = tokens[1:]
    for token in tokens:
        if token in entities_tokens:
            entity = token[1:-1] # remove <,>
            if entity == 'O':
                blabel = ilabel = entity
            else:
                blabel = f'B-{entity}'
                ilabel = f'I-{entity}'
            _len = len(current_entity)
            sequence_entities += [blabel] + [ilabel] * (_len - 1)
            current_entity.clear()
        elif token in (tokenizer.eos_token, tokenizer.pad_token):
            break
        else:
            current_entity.append(token)
    if length > 0:
        seq_len = len(sequence_entities)
        if seq_len > length:
            sequence_entities = sequence_entities[:length]
        elif seq_len < length:
            sequence_entities = sequence_entities + [fill_token] * (length - seq_len)
    return sequence_entities

In [25]:
def get_tokens(token_ids, tokenizer, entities):
    if isinstance(entities, dict):
        sentence = tokenizer.decode(token_ids)
        for ent, tok in entities.items():
            sentence = sentence.replace(ent, tok)
        return tokenizer.tokenize(sentence)
    else:
        return tokenizer.convert_ids_to_tokens(token_ids)

In [26]:
def get_trues_and_preds_entities(target_token_ids, predicted_token_ids,
                                tokenizer, entities, fill_token='O'):
    assert len(target_token_ids) ==  len(predicted_token_ids) # ensure batch size is the same
    all_target_entities = []
    all_predicted_entities = []
    entities_tokens = list(entities.values()) if isinstance(entities, dict) else entities
    for idx in range(len(target_token_ids)):
        # convert to tokens
        target_tokens = get_tokens(target_token_ids[idx], tokenizer, entities)
        predicted_tokens = get_tokens(predicted_token_ids[idx], tokenizer, entities)
        # convert to entities
        target_entities = get_entities_from_tokens(target_tokens, tokenizer, entities_tokens)
        predicted_entities = get_entities_from_tokens(predicted_tokens, tokenizer, entities_tokens, length=len(target_entities), fill_token=fill_token)
        # append
        all_target_entities.append(target_entities)
        all_predicted_entities.append(predicted_entities)
    return all_target_entities, all_predicted_entities

In [27]:
target_entities, predicted_entities = get_trues_and_preds_entities(target_token_ids, predicted_token_ids, tokenizer, entities=entities2tokens)

In [29]:
target_entities, predicted_entities

([['B-ORG', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O'],
  ['B-PER', 'I-PER', 'I-PER']],
 [['B-ORG', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O'],
  ['B-PER', 'I-PER', 'I-PER']])

# Seqeval

In [111]:
print(classification_report(target_entities, predicted_entities))

           precision    recall  f1-score   support

      PER       1.00      1.00      1.00         1
     MISC       1.00      1.00      1.00         2
      ORG       1.00      1.00      1.00         1

micro avg       1.00      1.00      1.00         4
macro avg       1.00      1.00      1.00         4



In [114]:
print(target_entities)

[['B-ORG', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O'], ['B-PER', 'I-PER', 'I-PER']]


In [115]:
print(predicted_entities)

[['B-ORG', 'O', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O'], ['B-PER', 'I-PER', 'I-PER']]
